In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os


In [5]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI = mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client.mongo_database